# 01 - PyCaret 基础教程

> 🎯 学习目标：掌握 PyCaret 的核心workflow，3行代码建立高性能baseline

---

## 📚 本节内容

1. PyCaret 简介与安装
2. 分类任务完整流程
3. 回归任务实战
4. 模型对比、调优、融合
5. 模型保存与部署
6. 实战案例：泰坦尼克生存预测

---

## 🚀 为什么选择 PyCaret？

**PyCaret vs 传统方法**：

```python
# 传统方法：50+ 行代码
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
# ... 大量代码 ...

# PyCaret：3 行代码
from pycaret.classification import *
setup(data, target='target')
best = compare_models()
```

---

## 1. 安装与导入

In [ ]:
# 安装 PyCaret（首次运行需要）
# !pip install pycaret

# 导入基础库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 设置显示选项
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8')

print("✅ 基础库导入成功")

---

## 2. 分类任务：泰坦尼克生存预测

### 2.1 加载数据

In [ ]:
# 使用 PyCaret 内置数据集
from pycaret.datasets import get_data

# 加载泰坦尼克数据集
data = get_data('titanic')

print(f"数据形状: {data.shape}")
data.head()

In [ ]:
# 数据概览
print("\n数据信息:")
print(data.info())

print("\n目标变量分布:")
print(data['Survived'].value_counts())

### 2.2 初始化 PyCaret 环境（核心步骤）

`setup()` 会自动完成：
- 数据类型推断
- 训练集/测试集划分
- 缺失值处理
- 类别特征编码
- 数据标准化
- 特征工程

In [ ]:
from pycaret.classification import *

# 初始化环境
clf = setup(
    data=data,
    target='Survived',
    session_id=123,           # 随机种子，保证可复现
    train_size=0.8,           # 训练集比例
    normalize=True,           # 标准化数值特征
    feature_selection=False,  # 是否特征选择
    verbose=False             # 减少输出
)

### 2.3 对比模型（自动训练15+模型）

这一步会自动：
- 训练15+个分类模型
- 10折交叉验证
- 计算多个评估指标
- 按准确率排序

In [ ]:
# 对比所有模型（返回最佳模型）
best_model = compare_models(
    n_select=1,          # 返回最佳的1个模型
    sort='Accuracy'      # 按准确率排序
)

### 2.4 查看最佳模型

In [ ]:
# 查看模型详情
print("最佳模型:")
print(best_model)

# 查看模型参数
print("\n模型参数:")
print(best_model.get_params())

### 2.5 模型调优（超参数优化）

In [ ]:
# 自动调优（使用随机搜索）
tuned_model = tune_model(
    best_model,
    n_iter=50,              # 尝试50组超参数
    optimize='Accuracy'     # 优化目标
)

### 2.6 可视化模型表现

In [ ]:
# 混淆矩阵
plot_model(tuned_model, plot='confusion_matrix')

In [ ]:
# ROC曲线
plot_model(tuned_model, plot='auc')

In [ ]:
# 特征重要性
plot_model(tuned_model, plot='feature')

### 2.7 模型融合（Ensemble）

In [ ]:
# 获取前3个最佳模型
top3 = compare_models(n_select=3, sort='Accuracy')

# Voting融合
blended = blend_models(top3)

# Stacking融合
stacked = stack_models(top3)

### 2.8 在测试集上评估

In [ ]:
# 在hold-out测试集上预测
predictions = predict_model(tuned_model)

print("测试集预测结果:")
predictions.head()

### 2.9 最终化模型（在全量数据上重新训练）

In [ ]:
# 在全量数据上训练最终模型
final_model = finalize_model(tuned_model)

print("✅ 最终模型训练完成")

### 2.10 保存模型

In [ ]:
# 保存模型
save_model(final_model, 'titanic_model')

print("✅ 模型已保存为 titanic_model.pkl")

### 2.11 加载模型并预测

In [ ]:
# 加载保存的模型
loaded_model = load_model('titanic_model')

# 对新数据预测
# new_predictions = predict_model(loaded_model, data=new_data)

print("✅ 模型加载成功")

---

## 3. 回归任务示例

### 3.1 加载回归数据

In [ ]:
# 加载波士顿房价数据
from pycaret.datasets import get_data
boston = get_data('boston')

print(f"数据形状: {boston.shape}")
boston.head()

### 3.2 回归任务完整流程

In [ ]:
from pycaret.regression import *

# 1. 初始化
reg = setup(
    data=boston,
    target='medv',
    session_id=123,
    normalize=True,
    verbose=False
)

# 2. 对比模型
best_reg = compare_models()

# 3. 调优
tuned_reg = tune_model(best_reg)

# 4. 可视化
plot_model(tuned_reg, plot='residuals')

# 5. 预测
predictions = predict_model(tuned_reg)

print("✅ 回归任务完成")

---

## 4. 高级功能

### 4.1 自定义指标

In [ ]:
# 添加自定义评估指标
from sklearn.metrics import make_scorer, f1_score

# 定义自定义指标
def custom_metric(y_true, y_pred):
    return f1_score(y_true, y_pred, average='weighted')

# 在 setup 中使用
# clf = setup(..., custom_metric=make_scorer(custom_metric))

### 4.2 特征工程选项

In [ ]:
# 更多特征工程选项
clf_advanced = setup(
    data=data,
    target='Survived',
    
    # 数据预处理
    normalize=True,                  # 标准化
    transformation=True,             # 特征变换
    transform_target=False,          # 目标变量变换
    
    # 特征工程
    feature_selection=True,          # 特征选择
    feature_interaction=True,        # 特征交互
    polynomial_features=True,        # 多项式特征
    
    # 采样
    fix_imbalance=True,             # 处理类别不平衡
    
    # 其他
    remove_outliers=True,           # 移除异常值
    session_id=123,
    verbose=False
)

---

## 5. 总结与最佳实践

### PyCaret 核心流程

```python
# 步骤1: 初始化（自动特征工程）
setup(data, target='target')

# 步骤2: 对比模型（自动训练15+模型）
best = compare_models()

# 步骤3: 调优（自动超参数优化）
tuned = tune_model(best)

# 步骤4: 融合（提升性能）
blended = blend_models(top_models)

# 步骤5: 评估与可视化
plot_model(tuned, plot='confusion_matrix')

# 步骤6: 最终化与保存
final = finalize_model(tuned)
save_model(final, 'my_model')
```

### 最佳实践

1. **快速baseline**: 使用 `compare_models()` 快速找到最佳模型
2. **分析特征**: 使用 `plot_model(..., plot='feature')` 查看特征重要性
3. **理解选择**: 分析为什么某个模型最好，指导后续手动调优
4. **模型融合**: 融合多个模型通常能提升1-3%性能
5. **交叉验证**: PyCaret默认使用10折交叉验证，结果可靠

### 优势与局限

**优势**：
- ✅ 极易上手，代码简洁
- ✅ 自动特征工程
- ✅ 快速对比多个模型
- ✅ 丰富的可视化

**局限**：
- ❌ 对于特殊业务需求，灵活性不如手动
- ❌ 深度学习支持有限
- ❌ 大数据集性能一般

---

## 🎯 练习任务

1. **任务1**: 使用 PyCaret 完成 `iris` 数据集的分类
2. **任务2**: 对比 PyCaret 和手动训练的模型性能差异
3. **任务3**: 使用 PyCaret 完成 Kaggle 泰坦尼克竞赛
4. **任务4**: 分析 PyCaret 自动生成的特征工程逻辑

---

## 📚 参考资源

- [PyCaret 官方文档](https://pycaret.gitbook.io/docs/)
- [PyCaret GitHub](https://github.com/pycaret/pycaret)
- [PyCaret 教程](https://pycaret.org/tutorial/)

---

**下一节**: `02_autosklearn_intro.ipynb` - Auto-sklearn 使用指南